In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})

#%%
# plot apical distance against Tc
df_super = pd.read_csv("/home/rhys/PhD/lattice/data/processed/super_apical.csv", index_col=0)

api_super = df_super["cu-o_a :"].values.T
plnr_super = df_super["cu-o_p :"].values.T
tc_super = df_super["tc :"].values.T

families = ["One", "Two", "Three"]


fig, ax = plt.subplots(figsize=(10, 7))

for fam in families:
    mask = np.asarray(df_super["layers :"] == fam).nonzero()
    ax.plot(api_super[mask], tc_super[mask],
            label=fam,
            marker="x",
            linestyle="None",
            )

ax.set_ylabel('Tc / K')
ax.set_xlabel('Apical / Å')
fig.tight_layout()
# plt.show()
fig.savefig("api-tc.pdf", bbox_inches='tight')

#%%
# plot c against tc
df_super = pd.read_csv("/home/rhys/PhD/lattice/data/processed/super_apical.csv", index_col=0)

c_super = df_super["latc :"].values.T
tc_super = df_super["tc :"].values.T

selection = "layers :"

# families = df_super[selection].unique()
families = ["One", "Two", "Three"]

fig, ax = plt.subplots(figsize = (10,7))
plt.set_cmap('tab10')

for fam in families:
    mask = np.asarray(df_super[selection]==fam).nonzero()
    ax.plot(c_super[mask], tc_super[mask], 
            label=fam,
            marker="x", 
            linestyle="None",
            )

ax.set_ylabel('Tc / K')
ax.set_xlabel('c / Å')
fig.tight_layout()
# plt.show()
fig.savefig("c-tc.pdf", bbox_inches='tight')


#%%
## plot a against tc
df_super = pd.read_csv("/home/rhys/PhD/lattice/data/processed/super_apical.csv", index_col=0)

a_super = df_super["lata :"].values.T
tc_super = df_super["tc :"].values.T

selection = "layers :"

# families = df_super[selection].unique()
families = ["One", "Two", "Three"]

fig, ax = plt.subplots( figsize = (10,7))
plt.set_cmap('tab10')

for fam in families:
    mask = np.asarray(df_super[selection]==fam).nonzero()
    ax.plot(a_super[mask], tc_super[mask], 
            label=fam,
            marker="x", 
            linestyle="None",
            )



ax.set_ylabel('Tc / K')
ax.set_xlabel('a* / Å')

fig.tight_layout()
# plt.show()
fig.savefig("a-tc.pdf", bbox_inches='tight')




#%%


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df_super = pd.read_csv("/home/reag2/PhD/first-year/apical/processed-data/super_apical.csv", index_col=0)

lata_super = df_super["lata :"].values.T
latc_super = df_super["latc :"].values.T
family_super = df_super["str3 :"].values

df_icsd = pd.read_csv("/home/reag2/PhD/first-year/apical/processed-data/icsd_cleaned.csv", index_col=0)

latc_icsd = df_icsd["latc :"].values
lata_icsd = df_icsd["lata :"].values
family_icsd = df_icsd["str3 :"].values

sqrt2 = np.sqrt(2)
ortho = lambda x: x/sqrt2 if x > 5 else x
compare = np.vectorize(lambda x: x/sqrt2 if x > 5 else x)
lata_super = compare(lata_super)
lata_icsd = compare(lata_icsd)

families = df_super["str3 :"].unique()
families.sort()

num = len(families)
rows = 4
cols = int((num+rows-1)/rows)

fig_2d, ax_2d = plt.subplots(cols, rows, squeeze=False, figsize=(cols*3,rows*3))
plt.rcParams.update({'font.size': 12})

for i, ftype in enumerate(families):
    j, k = divmod(i, rows)
    
    mask_super = np.where(family_super==ftype)
    a_super = lata_super[mask_super] 
    c_super = latc_super[mask_super] 
    
    mask_icsd = np.where(family_icsd==ftype)
    a_icsd = lata_icsd[mask_icsd]
    c_icsd = latc_icsd[mask_icsd]
    
    ax_2d[j,k].plot(a_super, c_super, 'o', markersize=5)
    ax_2d[j,k].plot(a_icsd, c_icsd, 'x')
    ax_2d[j,k].set_title(ftype)

ax_2d[0,0].set_ylabel("c / Å")
ax_2d[1,0].set_ylabel("c / Å")
ax_2d[2,0].set_ylabel("c / Å")
ax_2d[3,0].set_xlabel("a / Å")
ax_2d[3,0].set_ylabel("c / Å")
ax_2d[3,1].set_xlabel("a / Å")
ax_2d[3,2].set_xlabel("a / Å")
ax_2d[3,3].set_xlabel("a / Å")


fig_2d.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans

super_ = pd.read_csv("/home/rhys/PhD/lattice/data/processed/super_cleaned.csv")
icsd = pd.read_csv("/home/rhys/PhD/lattice/data/processed/icsd_cleaned.csv")

families = ["Tl1201", "Hg1201", "Tl1212", "Tl1223"]

fig, ax = plt.subplots(2,2, figsize=(10,10))
plt.rcParams.update({'font.size': 16})

for i, fam in enumerate(families):
    j, k = divmod(i,2)                   
    idx = (super_["str3 :"]==fam)
    X = super_.loc[idx, ["lata :", "latc :"]].values
    y_pred = KMeans(n_clusters=2, random_state=0).fit_predict(X)
    
    ax[j,k].scatter(X[:, 0], X[:, 1], c=y_pred)
    ax[j,k].set_title(fam)
    
ax[0,0].set_ylabel("c / Å")
ax[1,0].set_ylabel("c / Å")
ax[1,0].set_xlabel("a / Å")
ax[1,1].set_xlabel("a / Å")


fig.tight_layout()
plt.savefig("k-means.pdf", bbox_inches='tight')
plt.show()